In [10]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 8.4 MB/s eta 0:00:00


In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
print(tf.__version__)

2.12.0


In [3]:
fashion_mnist = keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [5]:
train_images = train_images/255.0
test_images = test_images/255.0

In [6]:
train_images[1].shape

(28, 28)

In [7]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [8]:
def build_model(hp):
  model = keras.Sequential([

      #any no. of conv 2D layer can be added
      keras.layers.Conv2D(
                filters= hp.Int('conv_1_filter', min_value= 32, max_value=128, step=16),
          kernel_size=hp.Choice('conv_1_kernel', values=[3,5]),
          activation='relu',
          input_shape=(28,28,1)
      ),

      keras.layers.Conv2D(
          filters= hp.Int('conv_2_filter', min_value= 32, max_value=64, step=16),
          kernel_size=hp.Choice('conv_2_kernel', values=[3,5]),
          activation='relu'
      ),

      keras.layers.Flatten(),

      keras.layers.Dense(
          units= hp.Int('dense_1_units', min_value=32, max_value=128, step=16),   #nodes in form of min max values here in dense
          activation= 'relu'
      ),

      keras.layers.Dense(10, activation='softmax')

  ])

  model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),

                loss = 'sparse_categorical_crossentropy',     #loss function mostly used this for classification/categorical problem
                metrics=['accuracy']
                )

  return model


In [11]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-11-2bbfbe7dae29>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [12]:
tuner_search = RandomSearch(build_model,
                            objective ='val_accuracy',
                            max_trials=5, directory ='output', project_name="MNIST Fashion")

In [14]:
tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)

Trial 5 Complete [00h 00m 43s]
val_accuracy: 0.8796666860580444

Best val_accuracy So Far: 0.903333306312561
Total elapsed time: 00h 03m 34s


In [17]:
# Get the best model and hyperparameters
best_model = tuner_search.get_best_models(num_models=1)[0]
best_hyperparameters = tuner_search.get_best_hyperparameters(num_trials=1)[0]

In [18]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 64)        1664      
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 64)        102464    
                                                                 
 flatten (Flatten)           (None, 25600)             0         
                                                                 
 dense (Dense)               (None, 32)                819232    
                                                                 
 dense_1 (Dense)             (None, 10)                330       
                                                                 
Total params: 923,690
Trainable params: 923,690
Non-trainable params: 0
_________________________________________________________________


In [23]:

# Print the best hyperparameters
print("Best Hyperparameters:")
print("Learning Rate:", best_hyperparameters.get('learning_rate'))
print("Conv1 Filter:", best_hyperparameters.get('conv_1_filter'))
print("Conv1 Kernel:", best_hyperparameters.get('conv_1_kernel'))
print("Conv2 Filter:", best_hyperparameters.get('conv_2_filter'))
print("Conv2 Kernel:", best_hyperparameters.get('conv_2_kernel'))
print("Dense1 Units:", best_hyperparameters.get('dense_1_units'))

Best Hyperparameters:
Learning Rate: 0.001
Conv1 Filter: 64
Conv1 Kernel: 5
Conv2 Filter: 64
Conv2 Kernel: 5
Dense1 Units: 32


In [24]:
# model fit

best_model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 10s 5ms/step - loss: 0.2096 - accuracy: 0.9229 - val_loss: 0.2609 - val_accuracy: 0.9085
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1770 - accuracy: 0.9345 - val_loss: 0.2653 - val_accuracy: 0.9098
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1501 - accuracy: 0.9446 - val_loss: 0.2932 - val_accuracy: 0.9100
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1294 - accuracy: 0.9520 - val_loss: 0.3157 - val_accuracy: 0.9078
Epoch 8/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1090 - accuracy: 0.9602 - val_loss: 0.3070 - val_accuracy: 0.9085
Epoch 9/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0934 - accuracy: 0.9651 - val_loss: 0.3599 - val_accuracy: 0.9067
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0807 - accuracy: 0.9702 - val_loss: 0.3458 - val_accura